In [ ]:
import sys
sys.path.append('../src')
from agent import EconomicAgent, CopAgent
from model import EconomicModel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
model = EconomicModel(num_econ_agents=30, initial_cops=0, width=10, height = 10, election_frequency=20, interaction_memory=20)
for i in range(1000):
    model.step()

In [ ]:
#the model data contains data collected at the model level
#mostly through counters that agents update when they do stuff
#this will show current model params or cumulative data
model_data = model.datacollector.get_model_vars_dataframe()

#agent data is collected at the agent level. Watch out because cops are still agents but don't have many params
#so NaN values are for cops, these can be dropped when calculating most stats
agent_data = model.datacollector.get_agent_vars_dataframe()


In [ ]:
agent_data.dropna()#this drops cop data from the agent-centered stats
mean_df = agent_data.groupby('Step').mean().reset_index()
df = pd.merge(model_data, mean_df)


## Hypothesis 1: Comparing policing levels with crime rates

In [ ]:
x = df['Step']  # Assuming 'Step' is your x-axis (you can replace it with the actual x-axis data)
y1 = df['num_cops']
y2 = df['num_crimes_committed']

# Plotting num_cops
fig, ax1 = plt.subplots(figsize=(10, 6))
ax1.plot(x, y1, color='b', linewidth = 0.1, label='num_cops')
ax1.set_xlabel('Step')
ax1.set_ylabel('num_cops', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Creating second y-axis and plotting num_crimes_committed
ax2 = ax1.twinx()
ax2.plot(x, y2, color='r', label='num_crimes_committed')
ax2.set_ylabel('num_crimes_committed', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Adding title and legend
plt.title('Number of Cops vs Number of Crimes Committed')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Display the plot
plt.grid(True)  # Optionally, add gridlines
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
df.columns

In [ ]:
x = df['Step']  # Assuming 'Step' is your x-axis (you can replace it with the actual x-axis data)
y1 = df['total_stolen']
y2 = df['total_trade_income']

# Plotting num_cops
fig, ax1 = plt.subplots(figsize=(10, 6))
ax1.plot(x, y1, color='b', linewidth = 1, label='total_stolen')
ax1.set_xlabel('Step')
ax1.set_ylabel('total_stolen', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Creating second y-axis and plotting num_crimes_committed
ax2 = ax1.twinx()
ax2.plot(x, y2, color='r', label='total_trade_income')
ax2.set_ylabel('total_trade_income', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Adding title and legend
plt.title('Number of Cops vs Number of Crimes Committed')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Display the plot
plt.grid(True)  # Optionally, add gridlines
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

## Hypothesis 2: Collective wealth, perceived crime threat, and level of policing

In [ ]:
plt.figure(figsize=(14, 7))

# to see if the avg individual's wealth would affect the number of police
plt.subplot(121)
ax1 = plt.gca()  
ax2 = ax1.twinx() 

ax1.plot(model_data['Step'], model_data['avg_wealth'], 'b-', label='Average Wealth')
ax2.plot(model_data['Step'], model_data['num_cops'], 'r-', label='Number of Cops')

ax1.set_xlabel('Step')
ax1.set_ylabel('Average Wealth', color='b')
ax2.set_ylabel('Number of Cops', color='r')
ax1.set_title('Average Wealth vs. Number of Cops Over Time')

ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# to see if the overall economy (collective wealth) would affect the number of police
plt.subplot(122)
ax3 = plt.gca()
ax4 = ax3.twinx() 

ax3.plot(model_data['Step'], model_data['total_wealth'], 'b-', label='Total Wealth')
ax4.plot(model_data['Step'], model_data['num_cops'], 'r-', label='Number of Cops')

ax3.set_xlabel('Step')
ax3.set_ylabel('Total Wealth', color='b')
ax4.set_ylabel('Number of Cops', color='r')
ax3.set_title('Total Wealth vs. Number of Cops Over Time')

ax3.legend(loc='upper left')
ax4.legend(loc='upper right')

plt.tight_layout()
plt.show()



In [ ]:
plt.figure(figsize=(14, 7))

# Average Wealth and Voting Outcome Over Time
plt.subplot(121)
ax1 = plt.gca() 
ax2 = ax1.twinx()
ax1.plot(model_data['Step'], model_data['avg_wealth'], 'b-', label='Average Wealth')
# # if tax is not decreases nor increases, we will not plot the vote outcome
non_zero_mask = model_data['vote_outcome'] != 0
ax2.plot(model_data['Step'][non_zero_mask], model_data['vote_outcome'][non_zero_mask], 'r-', label='Voting Outcome')

ax1.set_xlabel('Step')
ax1.set_ylabel('Average Wealth', color='b')
ax2.set_ylabel('Voting Outcome', color='r')

ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

ax1.set_title('Average Wealth and Voting Outcome Over Time')

# Total Wealth vs Voting Outcome Over Time
plt.subplot(122)
ax1 = plt.gca() 
ax2 = ax1.twinx()
ax1.plot(model_data['Step'], model_data['total_wealth'], 'b-', label='Total Wealth')
# if tax is not decreases nor increases, we will not plot the vote outcome
non_zero_mask = model_data['vote_outcome'] != 0
ax2.plot(model_data['Step'][non_zero_mask], model_data['vote_outcome'][non_zero_mask], 'r-', label='Voting Outcome')

ax1.set_xlabel('Step')
ax1.set_ylabel('Total Wealth', color='b')
ax2.set_ylabel('Voting Outcome', color='r')

ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

ax1.set_title('Total Wealth and Voting Outcome Over Time')

plt.tight_layout()
plt.show()